# A case study of MCLP in pySpatialOpt

In [3]:
# import packages
import logging
import sys
import arcpy
import pulp
from pyspatialopt.analysis import arcpy_analysis
from pyspatialopt.models import utilities
from pyspatialopt.models import covering
import os

In [10]:
# file path
scriptDir = os.getcwd()
print(scriptDir)
print(os.getcwd())

C:\Users\Huanfa Chen\Documents\GitHub\pyspatialopt\examples
C:\Users\Huanfa Chen\Documents\GitHub\pyspatialopt\examples


In [11]:
# load demand layer of polygons. Fields?
demand_polygon_fl = arcpy.MakeFeatureLayer_management(os.path.join(scriptDir, r"../sample_data/demand_polygon.shp")).getOutput(0)

In [27]:
# load facility service area layer
facility_service_areas_fl = arcpy.MakeFeatureLayer_management(os.path.join(scriptDir, 
        r"../sample_data/facility_service_areas.shp")).getOutput(0)

In [28]:
# describe the demand layer

print(demand_polygon_fl.__class__)
print(demand_polygon_fl.dataSource)
demand_polygon_fl.description

print(arcpy.Describe(demand_polygon_fl).shapeType)

# describe the layer fields
print(arcpy.Describe(demand_polygon_fl).fields)
# print the name of each element 
map(lambda x:x.name, arcpy.Describe(demand_polygon_fl).fields)

<class 'arcpy._mapping.Layer'>
C:\Users\Huanfa Chen\Documents\GitHub\pyspatialopt\sample_data\demand_polygon.shp
Polygon
[<geoprocessing describe field object object at 0x0000000004847210>, <geoprocessing describe field object object at 0x0000000004847AB0>, <geoprocessing describe field object object at 0x0000000004847BB0>, <geoprocessing describe field object object at 0x0000000004847310>]


[u'FID', u'Shape', u'GEOID10', u'Population']

In [29]:
# describe the facility service areas
print(arcpy.Describe(facility_service_areas_fl).shapeType)

# describe the layer fields
print(arcpy.Describe(facility_service_areas_fl).fields)
# print the name of each element 
map(lambda x:x.name, arcpy.Describe(facility_service_areas_fl).fields)

Polygon
[<geoprocessing describe field object object at 0x0000000004847C70>, <geoprocessing describe field object object at 0x0000000004847650>, <geoprocessing describe field object object at 0x0000000004847550>]


[u'FID', u'Shape', u'ORIG_ID']

So the `demand_polygon_fl` is a layer object. It can either reference layers in a map document (.mxd) and layers in a layer (.lyr) file.

Similarly, `facility_service_areas_fl` is a layer object consisting of polygons. 

In [13]:
# create binary coverage (polygon) dictionary structure
binary_coverage_polygon = arcpy_analysis.generate_binary_coverage(demand_polygon_fl, facility_service_areas_fl,
                                                                      "Population",
                                                                      "GEOID10", "ORIG_ID")

Binary coverage means that a demand unit can be either totally covered by a facility (meaning that serviceableDemand is equal to demand), or not covered by a facility. Also, a demand unit can be covered by more than one facilities.

Binary coverage of a facility `f1` to a polygonal demand unit `d1` is true when the service area of `f1` contains `d1`. 

The totalServiceableDemand is a sum of the serviceableDemand of all units.

In [52]:
# details in the binary_coverage_polygon
print(binary_coverage_polygon.keys())
print(binary_coverage_polygon['totalDemand'])
print(binary_coverage_polygon['totalServiceableDemand'])
print(binary_coverage_polygon['facilities'])
print(binary_coverage_polygon['type'])

# view the demand attribute. The core part!
dict_demand = binary_coverage_polygon['demand']
print(binary_coverage_polygon['demand'].__class__)
# first item
# This item in the dict corresponds to a demand unit. 
print(dict_demand.items()[0])
print(dict_demand.items()[1])
print(dict_demand.items()[2])
# print(dict(itertools.islice(dict_demand.iteritems(), 0, 2)))

['totalServiceableDemand', 'facilities', 'version', 'demand', 'totalDemand', 'type']
1029655.0
320453.0
{u'facility_service_areas': ['0', '1', '2', '3', '4', '5', '6', '7']}
{'type': 'binary', 'mode': 'coverage'}
<type 'dict'>
('49035101900', {'demand': 2497.0, 'serviceableDemand': 0, 'coverage': {u'facility_service_areas': {}}, 'area': 699021.0})
('49035110402', {'demand': 3653.0, 'serviceableDemand': 0, 'coverage': {u'facility_service_areas': {}}, 'area': 1747477.0})
('49035110401', {'demand': 3476.0, 'serviceableDemand': 3476.0, 'coverage': {u'facility_service_areas': {'4': 1}}, 'area': 1806460.0})


In [53]:
# create partial coverage (polygon) dictionary structure
partial_coverage_polygon = arcpy_analysis.generate_partial_coverage(demand_polygon_fl, facility_service_areas_fl,
                                                                      "Population",
                                                                      "GEOID10", "ORIG_ID")

Compare the results of binary_coverage and partial_coverage.

In [56]:
print("Total serviceable demand of BINARY coverage: ", binary_coverage_polygon['totalServiceableDemand'])
print("Total serviceable demand of PARTIAL coverage: ", partial_coverage_polygon['totalServiceableDemand'])
print("Yes. The total serviceable demand of partial coverage should exceed that of binary coverage")

('Total serviceable demand of BINARY coverage: ', 320453.0)
('Total serviceable demand of PARTIAL coverage: ', 562706.0)
Yes. The total serviceable demand of partial coverage should exceed that of binary coverage
